<a href="https://colab.research.google.com/github/Chang365/ESAA-25-1-assignments/blob/main/ESAA_25_2_0908_%EC%88%98%EC%83%81%EC%9E%91_%EB%A6%AC%EB%B7%B02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 대회명: SIIM‑FISABIO‑RSNA COVID‑19 Detection Challenge

## 목표: 흉부 X‑ray 이미지에서 COVID‑19 폐렴 존재 여부를 감지(detect)하고, 가능하다면 해당 병변 부위를 지역화(localize) 하는 모델 개발.

### 데이터 종류

- 이미지 종류: Chest Radiograph (흉부 엑스레이)
- 주석(annotation):폐렴 병변이 있는 경우 병변 부위 표시 포함, 여러 기관 및 전문가들이 주석 제공
- 참가 규모: 1,786명이 1,305팀으로, 82개국 참가
- 평가 지표: 정확도(accuracy), 정확한 localization 가능성이 중요 (binary classification + localization)

### 상위 솔루션 전략 요약 (1위: VinDr.ai DungNB 중심)

- 전처리: 이미지 품질이 다양하므로, 조정(pre‑processing) 중요: 해상도(normalization), 밝기/대비 조정, noise 제거.
병변 영역 외 배경(normal area) cropping 또는 마킹 기법 사용 가능.
- 데이터 증강: 더 다양한 조건(다른 노출, 방향, 지터, 회전, 랜덤 크랍 등) 사용하여 모델이 일반화 잘 하도록 함.
특히 병변이 상대적으로 작은 경우 localization을 놓치지 않도록 augmentation 중 병변이 잘려 나가지 않게 유의.
- 모델: CNN 기반의 strong backbone 사용 (예: EfficientNet, ResNet, DenseNet 등)
classification + localization 기능을 위한 멀티태스크 네트워크 또는 detection 네트워크 (예: U-Net 계열, FPN, Faster R‑CNN, RetinaNet 등) 혼합 사용 가능.
- 전이 학습: ImageNet 사전 훈련된 모델로 시작 → 이후 domain‑specific fine‑tuning (의료이미지로 더 트레이닝)
유사한 흉부 영상 데이터셋 미리 학습한 모델 활용 시 성능 상승 효과 있음.
- 손실 함수(Loss Function): 분류(Classification)에서는 binary cross‑entropy 또는 focal loss 사용
localization에서는 Dice Loss, IoU 기반 loss 병행하거나 조합.
false negative가 치명적일 수 있으므로 sensitivity 중심 조정도 고려됨.
- 앙상블 및 Test Time Augmentation: 	여러 모델 조합 (ensemble)으로 예측 안정성 확보.
TTA를 통해 테스트 이미지의 다양한 변형을 예측 → 평균 또는 가중 평균으로 결정.
- 검증(Validation): 전략	지역별, 기관별 데이터 분포 편차(different hospital, different acquisition device) 고려하여 검증 셋 분리.
Cross‑validation 또는 hold‑out set을 여러 세트로 구성 → 모델이 특정 조건에 과적합되지 않도록 함.
localization 정확도를 함께 평가함으로써 단순 분류 정확도만 높이는 게 아니라 실제 병변 위치 예측 성능도 중요시.

### 차별점 및 인사이트
- Localization 포함하는 태스크여서 단순 classification만 잘 해서는 안 되고, 병변 영역을 잘 추정할 수 있어야 함. 그래서 detection / segmentation 네트워크와 classification 네트워크의 균형이 중요했음.

- 의료영상 특성상 데이터 획득 조건이 다양 (장비, 노출, 해상도 등) → 일반화(generalization) 어려움이 존재함. 이를 개선하기 위해 전처리 + augmentation + 다양한 기관의 데이터 포함이 중요했음.

- False negative (병증 있는데도 감지 못하는 경우)의 비용이 높으므로, 민감도(sensitivity)를 높이는 방향으로 조정하는 게 승부처였음.

- 앙상블 + TTA가 상위권 팀에서 거의 기본 전략처럼 활용됨.